Preparing Data

In [1]:
import csv
from dbfread import DBF

def dbf_to_csv(dbf_filename, csv_filename):
    # Open the .dbf file using dbfread
    dbf = DBF(dbf_filename, load=True)
    
    # Open a .csv file to write the data
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        
        # Write the header (field names from the .dbf file)
        writer.writerow(dbf.field_names)
        
        # Write the rows of data
        for record in dbf:
            writer.writerow(record.values())

# Example usage
dbf_filename = 'seamounts.dbf'  # Replace with the path to your .dbf file
csv_filename = 'seamounts.csv'  # Replace with your desired output .csv file name

dbf_to_csv(dbf_filename, csv_filename)


In [1]:
import os
import requests
import pandas as pd

# Parameters
csv_file = "seamounts.csv"  # Path to the .dbf file
output_folder = "seamounts_galore"  # Folder to save downloaded images
tile_pixels = 800  # Width and height of the image in pixels
tile_width_deg = 1.0  # Width of the tile in degrees
api_base_url = "https://www.gmrt.org/services/ImageServer"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load CSV file
try:
    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} records from {csv_file}.")
except Exception as e:
    print(f"Error loading .csv file: {e}")
    exit()

# Process each record
for index, row in df.iterrows():
    try:
        # Extract data from the row
        file_name = str(row["PEAKID"])  # Use PEAKID as the file name
        center_lon = float(row["LONG"])  # Longitude from LONG
        center_lat = float(row["LAT"])  # Latitude from LAT

        # Calculate tile bounds
        minlatitude = center_lat - (tile_width_deg / 2)
        maxlatitude = center_lat + (tile_width_deg / 2)
        minlongitude = center_lon - (tile_width_deg / 2)
        maxlongitude = center_lon + (tile_width_deg / 2)

        # Prepare API parameters
        params = {
            "minlatitude": minlatitude,
            "maxlatitude": maxlatitude,
            "minlongitude": minlongitude,
            "maxlongitude": maxlongitude,
            "width": tile_pixels,
            "mask": "false",
            "download": "true",
        }

        # Construct file name
        output_file = os.path.join(output_folder, f"{file_name}.png")

        # Download the image
        print(f"Downloading image for PEAKID {file_name} at {center_lat}, {center_lon}...")
        response = requests.get(api_base_url, params=params)

        if response.status_code == 200:
            # Save the image
            with open(output_file, "wb") as file:
                file.write(response.content)
            print(f"Saved: {output_file}")
        else:
            print(f"Failed to download for PEAKID {file_name} - HTTP {response.status_code}")

    except Exception as e:
        print(f"Error processing row {index}: {e}")

print("Download complete.")


Loaded 20 records from seamounts.csv.
Saved: seamounts_galore/513718.0.png
Saved: seamounts_galore/514369.0.png
Saved: seamounts_galore/515210.0.png
Saved: seamounts_galore/515371.0.png
Saved: seamounts_galore/515426.0.png
Saved: seamounts_galore/516286.0.png
Saved: seamounts_galore/516446.0.png
Saved: seamounts_galore/516780.0.png
Saved: seamounts_galore/517485.0.png
Saved: seamounts_galore/517656.0.png
Saved: seamounts_galore/517746.0.png
Saved: seamounts_galore/518587.0.png
Saved: seamounts_galore/519028.0.png
Saved: seamounts_galore/519219.0.png
Saved: seamounts_galore/519304.0.png
Saved: seamounts_galore/519648.0.png
Saved: seamounts_galore/519807.0.png
Saved: seamounts_galore/519899.0.png
Saved: seamounts_galore/519970.0.png
Saved: seamounts_galore/520041.0.png
Download complete.
